<a href="https://colab.research.google.com/github/LucasDatilioCarderelli/Maratona_BehindTheCode_IBM21/blob/main/desafio-5/MBTC2021_df5_SONDA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# MBTC2021-df3_SONDA

## Imports

### installs

In [ ]:
# !pip install https://github.com/pandas-profiling/pandas-profiling/archive/master.zip
!pip install tpot
!pip install pycaret

### Libs

In [ ]:
import pandas                   as pd
import numpy                    as np
import matplotlib.pyplot        as plt
import seaborn                  as sns
# import warnings; warnings.simplefilter('ignore')
from pandas_profiling           import ProfileReport
from tpot                       import TPOTClassifier
from imblearn.under_sampling    import RandomUnderSampler, NearMiss
from imblearn.over_sampling     import RandomOverSampler, SMOTE
from sklearn.pipeline           import Pipeline
from sklearn.preprocessing      import OrdinalEncoder
from sklearn.preprocessing      import MinMaxScaler, Normalizer
from sklearn.model_selection    import train_test_split
from sklearn.tree               import DecisionTreeClassifier
from sklearn.metrics            import f1_score, classification_report

### Read csv's

In [ ]:
answer = pd.read_csv("https://raw.githubusercontent.com/maratonadev/desafio-5-2021/main/assets/data/ANSWERS.csv")
dataset = pd.read_csv("https://raw.githubusercontent.com/maratonadev/desafio-5-2021/main/assets/data/dataset.csv")

In [ ]:
answer.set_index("ID", inplace=True)
answer

,GENDER,SENIORCITIZEN,PARTNER,DEPENDENTS,TENURE,PHONESERVICE,MULTIPLELINES,INTERNETSERVICE,ONLINESECURITY,ONLINEBACKUP,DEVICEPROTECTION,TECHSUPPORT,STREAMINGTV,STREAMINGMOVIES,CONTRACT,PAPERLESSBILLING,PAYMENTMETHOD,MONTHLYCHARGES,TOTALCHARGES,CHURN
ID,,,,,,,,,,,,,,,,,,,,
6946,Female,0,No,No,3,Yes,No,No,No internet service,No internet service,No internet service,No internet service,No internet service,No internet service,Month-to-month,No,Electronic check,51.71,90.81,TO_FILL
2228,Male,0,No,No,47,Yes,No,No,No internet service,No internet service,No internet service,No internet service,No internet service,No internet service,Two year,No,Bank transfer (automatic),57.05,979.6,TO_FILL
3591,Male,0,Yes,No,62,Yes,Yes,Fiber optic,Yes,Yes,No,No,No,Yes,One year,No,Electronic check,235.56,6140.91,TO_FILL
1153,Male,0,No,Yes,1,Yes,Yes,Fiber optic,No,No,No,No,No,No,Month-to-month,Yes,Electronic check,142.25,142.25,TO_FILL
5227,Male,0,No,No,24,Yes,No,DSL,Yes,No,No,Yes,No,No,Month-to-month,No,Electronic check,105.40,1398.95,TO_FILL
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5478,Male,0,No,No,4,Yes,No,No,No internet service,No internet service,No internet service,No internet service,No internet service,No internet service,Month-to-month,No,Mailed check,23.41,75.76,TO_FILL
6297,Male,0,Yes,Yes,63,Yes,Yes,Fiber optic,No,Yes,Yes,No,Yes,Yes,Two year,Yes,Bank transfer (automatic),62.51,6554.96,TO_FILL
7952,Female,0,Yes,No,61,Yes,Yes,Fiber optic,No,No,Yes,Yes,Yes,Yes,Month-to-month,Yes,Electronic check,227.81,6872.81,TO_FILL


In [ ]:
dataset.set_index("ID", inplace=True)
dataset

,GENDER,SENIORCITIZEN,PARTNER,DEPENDENTS,TENURE,PHONESERVICE,MULTIPLELINES,INTERNETSERVICE,ONLINESECURITY,ONLINEBACKUP,DEVICEPROTECTION,TECHSUPPORT,STREAMINGTV,STREAMINGMOVIES,CONTRACT,PAPERLESSBILLING,PAYMENTMETHOD,MONTHLYCHARGES,TOTALCHARGES,CHURN
ID,,,,,,,,,,,,,,,,,,,,
3418,Female,0.0,No,No,26.0,Yes,No,DSL,No,Yes,Yes,No,No,No,Month-to-month,Yes,Credit card (automatic),131.55,1483.7,No
6865,Male,0.0,No,No,39.0,Yes,Yes,Fiber optic,No,No,No,No,No,No,Month-to-month,Yes,Credit card (automatic),79.61,2737.31,No
2799,Male,0.0,No,No,1.0,Yes,No,No,No internet service,No internet service,No internet service,No internet service,No internet service,No internet service,Month-to-month,No,Credit card (automatic),9.26,9.26,No
1584,Male,0.0,Yes,Yes,66.0,Yes,Yes,No,No internet service,No internet service,No internet service,No internet service,No internet service,No internet service,Two year,Yes,Credit card (automatic),42.14,1689.19,No
9997,Male,0.0,No,No,56.0,Yes,Yes,Fiber optic,No,Yes,No,No,Yes,Yes,Month-to-month,No,Electronic check,126.24,5608.69,No
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5586,Male,0.0,No,No,3.0,Yes,No,DSL,No,No,No,No,No,No,Month-to-month,NaN,Mailed check,25.14,121.39,No
4840,Male,0.0,No,No,7.0,Yes,Yes,Fiber optic,No,No,No,No,No,No,Month-to-month,No,Electronic check,200.74,647.14,Yes
7587,Male,0.0,No,No,18.0,Yes,Yes,Fiber optic,No,No,No,No,Yes,No,Month-to-month,No,Bank transfer (automatic),46.47,1500.07,No


## Analisys

In [ ]:
dataset.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4548 entries, 3418 to 5802
Data columns (total 20 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   GENDER            4520 non-null   object 
 1   SENIORCITIZEN     4523 non-null   float64
 2   PARTNER           4524 non-null   object 
 3   DEPENDENTS        4526 non-null   object 
 4   TENURE            4533 non-null   float64
 5   PHONESERVICE      4520 non-null   object 
 6   MULTIPLELINES     4523 non-null   object 
 7   INTERNETSERVICE   4524 non-null   object 
 8   ONLINESECURITY    4518 non-null   object 
 9   ONLINEBACKUP      4518 non-null   object 
 10  DEVICEPROTECTION  4524 non-null   object 
 11  TECHSUPPORT       4530 non-null   object 
 12  STREAMINGTV       4523 non-null   object 
 13  STREAMINGMOVIES   4523 non-null   object 
 14  CONTRACT          4525 non-null   object 
 15  PAPERLESSBILLING  4517 non-null   object 
 16  PAYMENTMETHOD     4520 non-null   objec

In [ ]:
answer.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1324 entries, 6946 to 7018
Data columns (total 20 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   GENDER            1324 non-null   object 
 1   SENIORCITIZEN     1324 non-null   int64  
 2   PARTNER           1324 non-null   object 
 3   DEPENDENTS        1324 non-null   object 
 4   TENURE            1324 non-null   int64  
 5   PHONESERVICE      1324 non-null   object 
 6   MULTIPLELINES     1324 non-null   object 
 7   INTERNETSERVICE   1324 non-null   object 
 8   ONLINESECURITY    1324 non-null   object 
 9   ONLINEBACKUP      1324 non-null   object 
 10  DEVICEPROTECTION  1324 non-null   object 
 11  TECHSUPPORT       1324 non-null   object 
 12  STREAMINGTV       1324 non-null   object 
 13  STREAMINGMOVIES   1324 non-null   object 
 14  CONTRACT          1324 non-null   object 
 15  PAPERLESSBILLING  1324 non-null   object 
 16  PAYMENTMETHOD     1324 non-null   objec

In [ ]:
# profile = ProfileReport(dataset)
# profile.to_file("your_report.html")

In [ ]:
dataset['TOTALCHARGES'] = dataset['TOTALCHARGES'].replace(' ', 0).astype(float)
answer['TOTALCHARGES'] = answer['TOTALCHARGES'].replace(' ', 0).astype(float)

## Preprocessing

In [ ]:
def prepocess(dataframe):
    df = dataframe.copy()
    
    to_drop = [# 'ID', 
            #    'GENDER', 'SENIORCITIZEN', 'PARTNER', 'DEPENDENTS',
            #    'PHONESERVICE', 'MULTIPLELINES','ONLINESECURITY', 
            #    'ONLINEBACKUP', 'DEVICEPROTECTION', 'TECHSUPPORT',
            #    'STREAMINGTV', 'STREAMINGMOVIES', 'PAYMENTMETHOD',
            #    'MONTHLYCHARGES', 'TOTALCHARGES', 'PAPERLESSBILLING',
            #    'TENURE', 'INTERNETSERVICE', 'CONTRACT', 
                ]

    df.drop(to_drop, axis=1, inplace=True)

    df.dropna(inplace=True)
    # df.fillna(value=0, inplace=True)

    df.replace({
        'No internet service': -1, 'No phone service': -1,
        'Yes': 1, 'No': 0
        }, inplace=True)
    df['INTERNETSERVICE'].replace({'DSL': 1, 'Fiber optic':2}, inplace=True)

    number_columns = df.select_dtypes(include=float).columns
    df[number_columns] = df[number_columns].astype(int)

    # df['TOTALCHARGES'] = df['TOTALCHARGES'].replace(' ', 0).astype(float)
    return df

# df = prepocess(dataset)
# df

KeyError: ignored

### Split

In [ ]:
X = df.drop("CHURN", axis=1)
y = df["CHURN"]

X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=.7, 
                                                    random_state=42)
X_train.shape, X_test.shape, y_train.shape, y_test.shape

### Encoding

In [ ]:
object_columns = X.select_dtypes(include=object).columns

# Encoding
enc = OrdinalEncoder().fit(X_train[object_columns])
X_train[object_columns] = enc.transform(X_train[object_columns])
X_test[object_columns] = enc.transform(X_test[object_columns])

# Normalizing
scaler = MinMaxScaler().fit(X_train)
X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)

# Balancing
sampler = RandomUnderSampler(random_state=42).fit(X_train, y_train)
X_train, y_train = sampler.fit_resample(X_train, y_train)
X_train.shape, y_train.shape

## Modeling

In [ ]:
DTC = DecisionTreeClassifier(random_state=42)
DTC.fit(X_train, y_train)
results = DTC.predict(X_test)
print('f1: ', f1_score(y_test, results), '\n',
      classification_report(y_test, results, digits=4))

### Feature Selection

In [ ]:
for n in range(0, len(DTC.feature_importances_)):
    print(f"{df.columns[n]:<17}{DTC.feature_importances_[n]*100:.2f}%")

In [ ]:
from sklearn.feature_selection import SelectKBest, f_classif

k_vs_score = []

for k in range(2,5,1):
  selector = SelectKBest(score_func=f_classif, k=k)

  X_train2 = selector.fit_transform(X_train, y_train)
  X_test2 = selector.transform(X_test)

  mdl = DecisionTreeClassifier(random_state=42)
  mdl.fit(X_train2, y_train)

  p = mdl.predict(X_test2)

  score = f1_score(y_test, p)
  print("k = {} - MSE = {}".format(k, score))

  k_vs_score.append(score)

In [ ]:
pd.Series(k_vs_score, index=range(2,18,1)).plot(figsize=(10,7))

In [ ]:
selector = SelectKBest(score_func=f_classif, k=3).fit(X_train, y_train)
mask = selector.get_support()
X.columns[mask]

In [ ]:
pd.Series(selector.scores_, index=X.columns).sort_values().tail(3).plot.barh()

### Tpot

In [ ]:
tpot = TPOTClassifier(generations=10, population_size=100, verbosity=2, 
                      scoring="f1", random_state=42, early_stop=4)
tpot.fit(X_train, y_train)
print(tpot.score(X_test, y_test))
tpot.export('tpot_pipeline.py')

In [ ]:
from sklearn.decomposition  import FastICA, PCA
from sklearn.ensemble       import RandomForestClassifier
from sklearn.ensemble       import GradientBoostingClassifier
from sklearn.naive_bayes    import GaussianNB
from sklearn.neighbors      import KNeighborsClassifier
from sklearn.pipeline       import make_pipeline, make_union
from xgboost                import XGBClassifier
from tpot.export_utils      import set_param_recursive
from tpot.builtins          import StackingEstimator

In [ ]:
# Average CV score on the training set was: 0.6277835595959895
# with ordinal-encoding
exported_pipeline = make_pipeline(
    StackingEstimator(estimator=RandomForestClassifier(bootstrap=True, criterion="entropy", max_features=0.1, min_samples_leaf=4, min_samples_split=9, n_estimators=100)),
    GaussianNB()
)
# Fix random state for all the steps in exported pipeline
set_param_recursive(exported_pipeline.steps, 'random_state', 42)

exported_pipeline.fit(X_train, y_train)
results = exported_pipeline.predict(X_test)
print('f1: ', f1_score(y_test, results), '\n',
      classification_report(y_test, results, digits=4))

In [ ]:
# Average CV score on the training set was: 0.8604245872676438
# with ordinal-encoding and over-sampling
exported_pipeline = GradientBoostingClassifier(
    learning_rate=0.1, max_depth=8, max_features=0.25,
    min_samples_leaf=5, min_samples_split=19, n_estimators=100, 
    subsample=0.85, random_state=42)

exported_pipeline.fit(X_train, y_train)
results = exported_pipeline.predict(X_test)
print('f1: ', f1_score(y_test, results), '\n',
      classification_report(y_test, results, digits=4))

In [ ]:
# Average CV score on the training set was: 0.8740417884811127
# With ordinal-encoding, over-sampling and feature-selection 
exported_pipeline = make_pipeline(
    FastICA(tol=0.6),
    PCA(iterated_power=1, svd_solver="randomized"),
    StackingEstimator(estimator=RandomForestClassifier(bootstrap=False, criterion="entropy", max_features=0.25, min_samples_leaf=7, min_samples_split=6, n_estimators=100)),
    KNeighborsClassifier(n_neighbors=3, p=1, weights="distance")
)
# Fix random state for all the steps in exported pipeline
set_param_recursive(exported_pipeline.steps, 'random_state', 42)

exported_pipeline.fit(X_train2, y_train2)
results = exported_pipeline.predict(X_test)
print('f1: ', f1_score(y_test, results), '\n',
      classification_report(y_test, results, digits=4))

In [ ]:
# Average CV score on the training set was: 0.7976823162958807
# with over-sampling after split
exported_pipeline = XGBClassifier(learning_rate=0.001, max_depth=4, min_child_weight=1, n_estimators=100, n_jobs=1, subsample=0.1, verbosity=0, random_state=42)

exported_pipeline.fit(X_train, y_train)
results = exported_pipeline.predict(X_test)
print('f1: ', f1_score(y_test, results), '\n',
      classification_report(y_test, results, digits=4))

In [ ]:

from sklearn.linear_model import SGDClassifier
from sklearn.pipeline import make_pipeline, make_union
from tpot.builtins import StackingEstimator
from xgboost import XGBClassifier

# Average CV score on the training set was: 0.7776879934103862
# with transforming in the rigth way
exported_pipeline = make_pipeline(
    StackingEstimator(estimator=SGDClassifier(alpha=0.001, eta0=1.0, fit_intercept=False, l1_ratio=1.0, learning_rate="invscaling", loss="squared_hinge", penalty="elasticnet", power_t=0.1)),
    XGBClassifier(learning_rate=0.001, max_depth=3, min_child_weight=12, n_estimators=100, n_jobs=1, subsample=0.15000000000000002, verbosity=0)
)
# Fix random state for all the steps in exported pipeline
set_param_recursive(exported_pipeline.steps, 'random_state', 42)

exported_pipeline.fit(X_train, y_train)
results = exported_pipeline.predict(X_test)
print('f1: ', f1_score(y_test, results), '\n',
      classification_report(y_test, results, digits=4))

### PyCaret

#### Initialize Setup

In [ ]:
from pycaret.classification import *
clf1 = setup(data = dataset, target = 'CHURN', session_id=42, 
            remove_multicollinearity=True, pca=True, normalize=True, 
            fix_imbalance=True, fix_imbalance_method=RandomUnderSampler(), train_size = 0.9
             )

,Description,Value
0,session_id,42
1,Target,CHURN
2,Target Type,Binary
3,Label Encoded,"No: 0, Yes: 1"
4,Original Data,"(4548, 20)"
5,Missing Values,True
6,Numeric Features,3
7,Categorical Features,16
8,Ordinal Features,False
9,High Cardinality Features,False


#### Compare models

In [ ]:
best_model = compare_models(sort='f1')

,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC,TT (Sec)
lr,Logistic Regression,0.7457,0.8408,0.8079,0.5093,0.6244,0.4467,0.4745,0.333
lda,Linear Discriminant Analysis,0.7374,0.8382,0.8107,0.4993,0.6178,0.4346,0.4644,0.031
ridge,Ridge Classifier,0.7369,0.0000,0.8032,0.4991,0.6154,0.4317,0.4601,0.022
nb,Naive Bayes,0.7540,0.8166,0.7501,0.5219,0.6151,0.4429,0.4590,0.024
gbc,Gradient Boosting Classifier,0.7364,0.8286,0.7799,0.4995,0.6082,0.4241,0.4487,1.165
rf,Random Forest Classifier,0.7447,0.8208,0.7520,0.5088,0.6065,0.4279,0.4463,0.922
qda,Quadratic Discriminant Analysis,0.7198,0.8299,0.8219,0.4799,0.6057,0.4109,0.4470,0.026
et,Extra Trees Classifier,0.7335,0.8086,0.7640,0.4945,0.6003,0.4139,0.4361,0.576
xgboost,Extreme Gradient Boosting,0.7313,0.8115,0.7622,0.4921,0.5979,0.4100,0.4322,9.945
lightgbm,Light Gradient Boosting Machine,0.7286,0.8179,0.7538,0.4883,0.5921,0.4024,0.4245,0.323


#### Create Models

In [ ]:
lr = create_model('lr')

,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
0,0.7610,0.8544,0.8318,0.5266,0.6449,0.4781,0.5065
1,0.7463,0.8703,0.8519,0.5111,0.6389,0.4616,0.4975
2,0.6976,0.7943,0.7037,0.4524,0.5507,0.3386,0.3574
3,0.7335,0.8585,0.8224,0.4944,0.6175,0.4319,0.4649
4,0.7751,0.8695,0.8598,0.5444,0.6667,0.5095,0.5399
5,0.7408,0.8150,0.7850,0.5030,0.6131,0.4320,0.4562
6,0.7384,0.8160,0.7944,0.5000,0.6137,0.4310,0.4574
7,0.7677,0.8781,0.8411,0.5357,0.6545,0.4922,0.5207
8,0.7482,0.8173,0.7477,0.5128,0.6084,0.4321,0.4488
9,0.7384,0.8477,0.8131,0.5000,0.6192,0.4367,0.4667


In [ ]:
gbc = create_model('gbc', random_state=42)

,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
0,0.7293,0.8283,0.7850,0.4884,0.6022,0.4134,0.4402
1,0.7244,0.8434,0.7593,0.4852,0.5921,0.3988,0.4216
2,0.6902,0.7818,0.6852,0.4431,0.5382,0.3209,0.3382
3,0.7042,0.8368,0.8224,0.4632,0.5926,0.3876,0.4271
4,0.7702,0.8721,0.8879,0.5367,0.6690,0.5089,0.5467
5,0.6993,0.8061,0.7664,0.4556,0.5714,0.3621,0.3912
6,0.7262,0.8097,0.7850,0.4855,0.6000,0.4089,0.4362
7,0.7579,0.8589,0.7757,0.5253,0.6264,0.4570,0.4760
8,0.7482,0.8205,0.7944,0.5120,0.6227,0.4467,0.4709
9,0.7311,0.8348,0.7944,0.4913,0.6071,0.4195,0.4475


In [ ]:
lda = create_model('lda')

,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
0,0.7439,0.8538,0.8131,0.5058,0.6237,0.4451,0.4739
1,0.7610,0.8700,0.8148,0.5301,0.6423,0.4747,0.4994
2,0.6976,0.7954,0.7500,0.4551,0.5664,0.3549,0.3811
3,0.7433,0.8562,0.8411,0.5056,0.6316,0.4527,0.4873
4,0.7579,0.8620,0.8598,0.5227,0.6502,0.4814,0.5163
5,0.7139,0.8093,0.7757,0.4716,0.5866,0.3872,0.4152
6,0.7139,0.8084,0.7850,0.4719,0.5895,0.3902,0.4200
7,0.7824,0.8811,0.8598,0.5542,0.6740,0.5219,0.5502
8,0.7506,0.8146,0.7477,0.5161,0.6107,0.4362,0.4524
9,0.7188,0.8445,0.8224,0.4783,0.6048,0.4094,0.4458


#### Tune Model

In [ ]:
tuned_lr = tune_model(lr, optimize='f1')

,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
0,0.7439,0.8536,0.8224,0.5057,0.6263,0.4479,0.4786
1,0.7634,0.8678,0.8426,0.5322,0.6523,0.4865,0.5161
2,0.6878,0.7974,0.7315,0.4438,0.5524,0.3341,0.3587
3,0.7311,0.8533,0.8224,0.4916,0.6154,0.4281,0.4617
4,0.7579,0.8666,0.8598,0.5227,0.6502,0.4814,0.5163
5,0.7164,0.8131,0.7757,0.4743,0.5887,0.3909,0.4185
6,0.7335,0.8081,0.7850,0.4941,0.6065,0.4204,0.4462
7,0.7775,0.8775,0.8598,0.5476,0.6691,0.5136,0.5433
8,0.7408,0.8159,0.7383,0.5032,0.5985,0.4171,0.4338
9,0.7262,0.8444,0.8224,0.4862,0.6111,0.4206,0.4553


In [ ]:
tuned_gbc = tune_model(gbc, optimize='f1')

,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
0,0.7707,0.8446,0.8037,0.5409,0.6466,0.4864,0.5073
1,0.7488,0.8501,0.7870,0.5152,0.6227,0.4465,0.4690
2,0.7293,0.7871,0.6944,0.4902,0.5747,0.3847,0.3972
3,0.7384,0.8487,0.8411,0.5000,0.6272,0.4451,0.4809
4,0.7555,0.8668,0.8131,0.5210,0.6350,0.4642,0.4902
5,0.7335,0.8184,0.7757,0.4940,0.6036,0.4174,0.4415
6,0.7457,0.8079,0.7570,0.5094,0.6090,0.4311,0.4497
7,0.7897,0.8750,0.8037,0.5695,0.6667,0.5195,0.5360
8,0.7506,0.8141,0.7196,0.5168,0.6016,0.4271,0.4395
9,0.7531,0.8442,0.7850,0.5185,0.6245,0.4518,0.4734


In [ ]:
tuned_lda = tune_model(lda, optimize='f1')

,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
0,0.7585,0.8610,0.8224,0.5238,0.6400,0.4715,0.4987
1,0.7561,0.8669,0.8241,0.5235,0.6403,0.4693,0.4970
2,0.7024,0.7907,0.7500,0.4602,0.5704,0.3622,0.3875
3,0.7433,0.8532,0.8224,0.5057,0.6263,0.4472,0.4780
4,0.7579,0.8663,0.8505,0.5230,0.6477,0.4788,0.5117
5,0.7213,0.8092,0.7944,0.4802,0.5986,0.4044,0.4345
6,0.7139,0.8064,0.7850,0.4719,0.5895,0.3902,0.4200
7,0.7653,0.8783,0.8411,0.5325,0.6522,0.4882,0.5173
8,0.7482,0.8154,0.7383,0.5130,0.6054,0.4291,0.4445
9,0.7335,0.8462,0.8411,0.4945,0.6228,0.4375,0.4745


#### Emsable

In [ ]:
blender = blend_models(estimator_list = [tuned_lr, tuned_gbc, tuned_lda], optimize='f1')

,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
0,0.7634,0.8539,0.8224,0.5301,0.6447,0.4795,0.5055
1,0.7585,0.8679,0.8426,0.5260,0.6477,0.4786,0.5093
2,0.6829,0.7889,0.7130,0.4375,0.5423,0.3204,0.3428
3,0.7384,0.8582,0.8318,0.5000,0.6246,0.4423,0.4761
4,0.7628,0.8706,0.8692,0.5284,0.6572,0.4919,0.5276
5,0.7213,0.8095,0.7757,0.4798,0.5929,0.3984,0.4250
6,0.7286,0.8103,0.7944,0.4885,0.6050,0.4157,0.4442
7,0.7775,0.8815,0.8505,0.5482,0.6667,0.5111,0.5389
8,0.7482,0.8153,0.7477,0.5128,0.6084,0.4321,0.4488
9,0.7286,0.8446,0.8505,0.4892,0.6212,0.4327,0.4730


In [ ]:
stacker = stack_models(estimator_list = [tuned_lr, tuned_gbc, tuned_lda], optimize='f1')

,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
0,0.7463,0.8486,0.8131,0.5088,0.6259,0.4490,0.4773
1,0.7634,0.8659,0.8333,0.5325,0.6498,0.4839,0.5116
2,0.6902,0.7938,0.7315,0.4463,0.5544,0.3377,0.3619
3,0.7335,0.8559,0.8224,0.4944,0.6175,0.4319,0.4649
4,0.7555,0.8690,0.8692,0.5196,0.6503,0.4801,0.5177
5,0.7384,0.8115,0.7570,0.5000,0.6022,0.4192,0.4393
6,0.7286,0.8155,0.7850,0.4884,0.6022,0.4127,0.4395
7,0.7775,0.8795,0.8505,0.5482,0.6667,0.5111,0.5389
8,0.7531,0.8186,0.7570,0.5192,0.6160,0.4431,0.4603
9,0.7359,0.8459,0.8318,0.4972,0.6224,0.4385,0.4729


#### Predict

In [ ]:
best = automl(optimize='f1')
best

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=1000,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=42, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [ ]:
pred_holdouts = predict_model(best)
pred_holdouts.head()

,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
0,Logistic Regression,0.7429,0.8578,0.777,0.5567,0.6486,0.4545,0.4702


,Component_1,Component_2,Component_3,Component_4,Component_5,Component_6,Component_7,Component_8,Component_9,Component_10,Component_11,Component_12,Component_13,Component_14,Component_15,Component_16,Component_17,Component_18,Component_19,Component_20,Component_21,Component_22,Component_23,Component_24,CHURN,Label,Score
0,1.163081,1.210622,0.138257,0.109895,-0.749634,0.571720,-0.204147,-0.838300,0.380595,0.602077,-0.899890,0.041814,-0.275407,0.909763,0.845767,-0.654464,-0.257889,-0.011085,-0.228538,-0.548018,0.029740,0.161124,0.004571,-0.002584,No,Yes,0.5948
1,-1.572866,0.247049,0.836381,-1.233560,0.275300,-0.786519,-0.287742,0.688462,0.706483,0.925040,0.296096,-0.635280,0.041751,0.237074,-0.322779,0.646174,-0.576547,-0.082881,-0.469173,-0.138120,0.408409,-0.402912,-0.089536,-0.001181,Yes,Yes,0.5130
2,-1.281207,1.045324,0.239290,1.458763,0.109753,0.500042,-0.383630,0.393198,0.174413,0.227864,0.279668,-0.143255,0.131397,-0.404135,-0.968525,-0.170671,-0.120218,-0.077845,0.024956,0.261686,-0.599459,0.122871,-0.072306,0.004720,Yes,Yes,0.8537
3,1.959355,-0.608962,0.641059,-0.763164,0.367898,1.167293,-0.367465,0.031046,-0.418292,0.468313,0.614096,-0.615549,-0.144845,0.683006,0.637239,0.131415,0.124417,1.144993,-0.189700,0.155021,0.016280,-0.293246,-0.076468,-0.000439,No,No,0.9588
4,3.760930,-0.030326,-0.574672,-0.589352,0.151873,-0.671014,0.885501,-0.363211,0.365716,0.737256,0.675438,-0.311981,-0.254719,-0.625613,0.297122,-0.121279,-0.197918,0.089245,0.749654,0.451792,0.186545,-0.085326,0.383099,-0.027814,No,No,0.8584


In [ ]:
new_data = answer.copy()
new_data.drop(['CHURN'], axis=1, inplace=True)
predict_new = predict_model(best, data=new_data)
predict_new

,GENDER,SENIORCITIZEN,PARTNER,DEPENDENTS,TENURE,PHONESERVICE,MULTIPLELINES,INTERNETSERVICE,ONLINESECURITY,ONLINEBACKUP,DEVICEPROTECTION,TECHSUPPORT,STREAMINGTV,STREAMINGMOVIES,CONTRACT,PAPERLESSBILLING,PAYMENTMETHOD,MONTHLYCHARGES,TOTALCHARGES,Label,Score
ID,,,,,,,,,,,,,,,,,,,,,
6946,Female,0,No,No,3,Yes,No,No,No internet service,No internet service,No internet service,No internet service,No internet service,No internet service,Month-to-month,No,Electronic check,51.71,90.81,No,0.5460
2228,Male,0,No,No,47,Yes,No,No,No internet service,No internet service,No internet service,No internet service,No internet service,No internet service,Two year,No,Bank transfer (automatic),57.05,979.60,No,0.9724
3591,Male,0,Yes,No,62,Yes,Yes,Fiber optic,Yes,Yes,No,No,No,Yes,One year,No,Electronic check,235.56,6140.91,No,0.7586
1153,Male,0,No,Yes,1,Yes,Yes,Fiber optic,No,No,No,No,No,No,Month-to-month,Yes,Electronic check,142.25,142.25,Yes,0.8778
5227,Male,0,No,No,24,Yes,No,DSL,Yes,No,No,Yes,No,No,Month-to-month,No,Electronic check,105.40,1398.95,No,0.6812
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5478,Male,0,No,No,4,Yes,No,No,No internet service,No internet service,No internet service,No internet service,No internet service,No internet service,Month-to-month,No,Mailed check,23.41,75.76,No,0.6385
6297,Male,0,Yes,Yes,63,Yes,Yes,Fiber optic,No,Yes,Yes,No,Yes,Yes,Two year,Yes,Bank transfer (automatic),62.51,6554.96,No,0.7604
7952,Female,0,Yes,No,61,Yes,Yes,Fiber optic,No,No,Yes,Yes,Yes,Yes,Month-to-month,Yes,Electronic check,227.81,6872.81,Yes,0.6022


## Submiting

In [ ]:
predict_new.drop("Score", axis=1, inplace=True)
predict_new.rename({"Label":"CHURN"}, axis=1, inplace=True)

In [ ]:
predict_new

,GENDER,SENIORCITIZEN,PARTNER,DEPENDENTS,TENURE,PHONESERVICE,MULTIPLELINES,INTERNETSERVICE,ONLINESECURITY,ONLINEBACKUP,DEVICEPROTECTION,TECHSUPPORT,STREAMINGTV,STREAMINGMOVIES,CONTRACT,PAPERLESSBILLING,PAYMENTMETHOD,MONTHLYCHARGES,TOTALCHARGES,CHURN
ID,,,,,,,,,,,,,,,,,,,,
6946,Female,0,No,No,3,Yes,No,No,No internet service,No internet service,No internet service,No internet service,No internet service,No internet service,Month-to-month,No,Electronic check,51.71,90.81,No
2228,Male,0,No,No,47,Yes,No,No,No internet service,No internet service,No internet service,No internet service,No internet service,No internet service,Two year,No,Bank transfer (automatic),57.05,979.60,No
3591,Male,0,Yes,No,62,Yes,Yes,Fiber optic,Yes,Yes,No,No,No,Yes,One year,No,Electronic check,235.56,6140.91,No
1153,Male,0,No,Yes,1,Yes,Yes,Fiber optic,No,No,No,No,No,No,Month-to-month,Yes,Electronic check,142.25,142.25,Yes
5227,Male,0,No,No,24,Yes,No,DSL,Yes,No,No,Yes,No,No,Month-to-month,No,Electronic check,105.40,1398.95,No
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5478,Male,0,No,No,4,Yes,No,No,No internet service,No internet service,No internet service,No internet service,No internet service,No internet service,Month-to-month,No,Mailed check,23.41,75.76,No
6297,Male,0,Yes,Yes,63,Yes,Yes,Fiber optic,No,Yes,Yes,No,Yes,Yes,Two year,Yes,Bank transfer (automatic),62.51,6554.96,No
7952,Female,0,Yes,No,61,Yes,Yes,Fiber optic,No,No,Yes,Yes,Yes,Yes,Month-to-month,Yes,Electronic check,227.81,6872.81,Yes


In [ ]:
predict_new.to_csv("ANSWER.CSV")